In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
!pip install transformers[sentencepiece]
!pip  install transformers 
!pip install emoji
!pip install unidecode
!pip install tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 23.9 MB/s 
     |████████████████████████████████| 86 kB 7.9 MB/s 
     |████████████████████████████████| 6.6 MB 58.9 MB/s 
     |████████████████████████████████| 596 kB 90.4 MB/s 
     |████████████████████████████████| 1.2 MB 67.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 26.2 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=e9e9d366749ee08a9196b9b2ed2b978bca4acb439fbf266d6172aa63e246defe
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b

In [4]:
import torch 
from transformers import pipeline , AutoTokenizer , AutoModelForSequenceClassification,  AutoModelForTokenClassification
import pandas as pd
from csv import writer
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModelForTokenClassification, pipeline
import keras 
import csv
import tensorflow_text

In [5]:
from emoji import UNICODE_EMOJI
import unidecode
import re

class clean_data():
    # remove URLs   
    def remove_URL(self,text):
        url = re.compile(r'https?://\S+|www\.\S+')
        return url.sub(r'',text)

    # remove htmls
    def remove_html(self,text):
        html=re.compile(r'<.*?>')
        return html.sub(r'',text)
    
    # remove punct
    def remove_punct(self,text):
        punc = '''!()[]{};:"\,<>./?@#$%^&*_~'''
        for ele in text:
            if ele in punc:
                text= text.replace(ele, "")
        return text

    # remove other ...
    def remove_other (self,text) : 
        text = text.rstrip()
        text = re.sub(' +', ' ', text)
        text = re.sub('\[.*?\]', '', text)
        text = re.sub('<.*?>+', '', text)
        text = re.sub('\n', '', text)
        #text = re.sub('\w*\d\w*', '', text)
        text = re.sub(',', '', text)
        return(text)

    # lower   
    def lower_text(self, text):
        return text.lower()

    # remove emojis
    def remove_emoji(self, text):
        to_replace = UNICODE_EMOJI["en"]
        result = text
        for x in to_replace:
            result = result.replace(x, "")
        return result

    # remove accents
    def remove_accents(self, text):
        return unidecode.unidecode(text)

    # clean text
    def clean(self,text) : 
        text = self.lower_text(text)
        text = self.remove_URL(text)
        text = self.remove_html(text)
        text = self.remove_punct(text)
        text = self.remove_emoji(text)
        text = self.remove_other(text)
        return text

In [13]:
def read_data(path):
  df=pd.read_csv(path)
  return df
all=read_data("/content/drive/MyDrive/Trend_analysis/fb_data.csv")
all.comment=all.comment.astype(str)

,2022-06-02 17:58:20,2022-06-02 18:11:20,C'est tout à votre honneur ! Nous devrions avoir plus de députés comme vous. Les autres sont à jeter aux chiottes.,6
0,2022-06-02 17:11:22,2022-06-02 18:11:24,pour la créolisation de la france 😬,6
1,2022-06-02 17:51:22,2022-06-02 18:11:24,Pas une voix pour le Premier sinistre...,6
2,2022-06-02 17:11:22,2022-06-02 18:11:24,Le stalinien dans toute sa splendeur .,6
3,2022-06-02 18:00:22,2022-06-02 18:11:24,Excellent titre!,6
4,2022-06-02 17:30:22,2022-06-02 18:11:24,"Vous avez raison, Melenchon-Le Pen-Zemmour son...",6


In [7]:

def label_filter(model, comment, labels):
    label=False
    for word in comment.split():
      if word in labels:
        label=True
    if label==True:
        one_name=True
        for x in model(comment):
            if (x['entity_group']== 'ORG' or x['entity_group']== 'PER') and x['word'] not in labels:
                one_name=False
        if one_name==True:
            return True
        else:
            return False
    else:
        return False

def run_filter(model, comment):
    partis={   
    1: ["extreme gauche", "lutte ouvriere", "anticapitaliste","lo", "npa", "nathalie arthaud", "nathalie", "arthaud", "philippe poutou", "poutou"],
    2: ["france insoumise", "union populaire", "fi", "up", "jean luc melenchon","melenchon", "jlmelenchon", "parti communiste", "communiste", "pcf", "fabien roussel", "roussel", "europe ecologie les verts", "les verts", "ecologie les verts", "eelv", "yannick jadot", "yannick", "jadot", "parti socialiste", "ps", "anne hidalgo", "hidalgo", "anne", "nupes"],
    3: ["macron", "emmanuel", "emmanuel macron","la republique en marche", "republique en marche","lrem","renaissance", "larem", "rem", "lrm", "modem", "mdm", "françois bayrou", "bayrou", "horizons", "edouard philippe", "edouard"],
    4: ["les republicains", "lr", "valerie pecresse", "valerie", "pecresse", "christian jacob", "jacob", "L'union des democrates et independants", "udi", "jean louis borloo", "borloo", "jean christophe lagarde", "lagarde", "aurelien pradie", "aurelien", "pradie", "olivier marleix","marleix"],
    5: ["debout la france", "dlf", "nicolas dupont Aignan", "nicolas dupont", "patriotes", "florian philippot", "florian", "philippot"],
    6: ["marine le pen", "marine lepen", "marine", "lepen", "rassemblement national" ,"mlp", "rn", "le pen", "jordan bardella", "jordan", "bardella"],
    7: ["reconquete", "eric zemmour", "zemmour", 'zemmerde',"marion marechal", "marion", "marechal", "nicolas bay", "bay", "guillaume peltier", "guillaume", "peltier", "stanislas rigault", "stanislas"]
}
    comment = clean_data().remove_accents(comment)
    partis_nb=0
    resultat=[]
    for p in partis.items():
        if label_filter(model, comment, p[1])== True:
            partis_nb+=1
            resultat.append(p[0])
    if partis_nb==1:
        return resultat[0]
    else:
        return 0


In [8]:
MODEL_NAME="Jean-Baptiste/camembert-ner"
tokenizer0 = AutoTokenizer.from_pretrained(MODEL_NAME)
model0 = AutoModelForTokenClassification.from_pretrained(MODEL_NAME)
filter= pipeline('ner', model=model0, tokenizer=tokenizer0, aggregation_strategy="simple")

Downloading:   0%|          | 0.00/269 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/210 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

In [9]:
def main(df):
      for i in range(len(df)):
        id_groupe=run_filter(filter, clean_data().clean(df['comment'][i]))
        if id_groupe!=0:
            val = [df['date_publication'][i], df['date_extraction'][i], df['comment'][i], df['id_source'][i], id_groupe]
            print(val)
            with open("/content/drive/MyDrive/Trend_analysis/data_filtre.csv", 'a', encoding='UTF8') as f:
                    writer = csv.writer(f)
                    writer.writerow(val)


In [10]:
main(all)

['2022-05-31 15:33:04+00:00', '2022-06-01', 'ça parle de #racaille ("racaille") mais ça n\'a JAMAIS parlé de ce CRIMINEL de 72 ans ayant foncé EXPRÈS sur un père de famille pour simple raison #raciste. voilà où mènent les idées de #Reconquete et du #RassemblementNational chez certains:\n\nhttps://t.co/wnAqgAS9H2', 68, 7]
['2022-05-30 12:43:16+00:00', '2022-06-01', "Curieux d'appeler un parti #RassemblementNational quand on ne veut pas rassembler. Le #RN devrait changer de nom pour Désillusion Nationale car à chaque élection c'est la déception. \n#ReconqueteLegislatives2022", 68, 6]
['2022-05-27 19:45:35+00:00', '2022-06-01', '🔴👉La justice qui suspend l’autorisation du burkini dans les piscines municipales est une bonne leçon de République aux élus EELV! Pourvu que cela continue! #RassemblementNational 🇫🇷', 68, 2]
['2022-05-25 11:47:27+00:00', '2022-06-01', 'On est à deux semaines des élections et seul la #NUPES a produit un programme.\n\nLes autres (#EnsembleMajoritePresidentielle , #R